In [1]:
import sys
sys.path.append("..")
%load_ext autoreload
%autoreload 2


In [2]:
from src.dvip import DVIP_Base
from utils.dataset import *
from src.generative_functions import *
from src.layers import VIPLayer
from src.likelihood import *
from utils.metrics import MetricsClassification, MetricsRegression
import torch
from utils.pytorch_learning import fit_with_metrics
from torch.utils.data import DataLoader

seed = 2147483647

In [3]:
torch.cuda.is_available()
device = torch.device("cpu")  

In [4]:
dataset = MNIST_Dataset()
mc = MultiClass(
            num_classes=10, device=device, dtype=torch.float64
        )
likelihood = BroadcastedLikelihood(mc)
metrics = MetricsClassification
train_dataset, train_test_dataset, test_dataset = dataset.get_split(0.1, seed)


Number of samples:  60000
Input dimension:  784
Label dimension:  1
Labels mean value:  0
Labels standard deviation:  1


In [5]:
gen1 = BayesianConvNN2(20, (28, 28), 1, activation = torch.tanh, device=device)

In [6]:
layer = VIPLayer(
                gen1,
                num_regression_coeffs=20,
                input_dim=784,
                output_dim=1,
                add_prior_regularization=False,
                mean_function=None,
                log_layer_noise=None,
                seed=seed,
                dtype=torch.float64,
                device=device,
            )

In [7]:
dvip = DVIP_Base(likelihood, [layer], len(train_dataset), device = device)

In [8]:
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
train_test_loader = DataLoader(train_test_dataset, batch_size=100)
val_loader = DataLoader(test_dataset, batch_size=100)

In [9]:
# Define optimizer and compile model
opt = torch.optim.Adam(dvip.parameters(), lr=0.001)

# Perform training
train_hist, val_hist = fit_with_metrics(
    dvip,
    train_loader,
    opt,
    metrics,
    val_generator=val_loader,
    epochs=100,
    device=device,
)

Training :   2%|▏         | 2/100 [13:01<10:37:57, 390.59s/epoch, loss_train=2.74e+5, nll_train=0.696, error_train=0.901, loss_val=9.13e+3, nll_val=0.695, error_val=0.902]


KeyboardInterrupt: 